<a href="https://colab.research.google.com/github/nghess/fractal-tools/blob/main/2D_Fractal_Boxcount_Single_D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
import numpy as np
import cv2
from os import listdir
from os.path import isfile, join

In [65]:
from google.colab import drive
drive.mount('/content/drive')
mypath = '/content/drive/My Drive/fractal_test_images/solid/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Import and Process Images

In [66]:
# Import and Process Images
filenames = sorted([f for f in listdir(mypath) if isfile(join(mypath, f))])
images = []

# Iterate through folder
for i in range(len(filenames)):
  image = cv2.imread(mypath+filenames[i], 0)
  #image = cv2.resize(image, None, fx=1, fy=1, interpolation=cv2.INTER_AREA)
  # Add image to list
  images.append(image)
  print(f'{filenames[i]} - {np.shape(image)}') # Print filename and dimensions

s2_d1.0.png - (915, 915)
s2_d1.1.png - (915, 915)
s2_d1.2.png - (915, 915)
s2_d1.3.png - (915, 915)
s2_d1.5.png - (915, 915)
s2_d1.6.png - (915, 915)
s2_d1.8.png - (915, 915)
s2_d1.9.png - (915, 915)
s2_d2.0.png - (915, 915)


# Box Counting Function

In [67]:
def fractal_dimension(Z, threshold=0.9):

    # Only for 2d image
    assert(len(Z.shape) == 2)

    # From https://github.com/rougier/numpy-100 (#87)
    def boxcount(Z, k):
        S = np.add.reduceat(
            np.add.reduceat(Z, np.arange(0, Z.shape[0], k), axis=0),
                               np.arange(0, Z.shape[1], k), axis=1)

        # We count non-empty (0) and non-full boxes (k*k)
        return len(np.where((S > 0) & (S < k*k))[0])

    # Transform Z into a binary array
    Z = (Z < threshold)

    # Minimal dimension of image
    p = min(Z.shape)

    # Greatest power of 2 less than or equal to p
    n = 2**np.floor(np.log(p)/np.log(2))

    # Extract the exponent
    n = int(np.log(n)/np.log(2))

    # Build successive box sizes (from 2**n down to 2**1)
    sizes = 2**np.arange(n, 1, -1)

    # Actual box counting with decreasing size
    counts = []
    for size in sizes:
        counts.append(boxcount(Z, size))

    # Fit the successive log(sizes) with log (counts)
    coeffs = np.polyfit(np.log(sizes), np.log(counts), 1)
    return -coeffs[0]

# Box Counting Results

In [72]:
# Thresholded Images
for i in range(len(images)):
  D = fractal_dimension(np.asarray(images[i]))
  print(f'{filenames[i]}: D~{D:.2f}')

s2_d1.0.png: D~1.18
s2_d1.1.png: D~1.25
s2_d1.2.png: D~1.32
s2_d1.3.png: D~1.40
s2_d1.5.png: D~1.59
s2_d1.6.png: D~1.69
s2_d1.8.png: D~1.83
s2_d1.9.png: D~1.88
s2_d2.0.png: D~1.91
